<a href="https://colab.research.google.com/github/WACoggins1/HCGNNChecker/blob/main/HCGNN_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import MessagePassing
import networkx as nx
import random

def generate_random_graph_with_cycle(num_nodes):
    # 1. Create a Hamiltonian cycle
    cycle_nodes = list(range(num_nodes))
    random.shuffle(cycle_nodes)

    # 2. Build a NetworkX graph with the forced cycle
    G = nx.Graph()
    G.add_nodes_from(cycle_nodes)
    for i in range(num_nodes):
        # Add edge between consecutive nodes in cycle
        u = cycle_nodes[i]
        v = cycle_nodes[(i+1) % num_nodes]
        G.add_edge(u, v)

    # 3. Randomly add extra edges
    extra_edges = int(num_nodes * 1.5)  # You can tweak
    for _ in range(extra_edges):
        u = random.randint(0, num_nodes-1)
        v = random.randint(0, num_nodes-1)
        if u != v:
            G.add_edge(u, v)

    # The forced cycle is known:
    cycle_edges = set()
    for i in range(num_nodes):
        u = cycle_nodes[i]
        v = cycle_nodes[(i+1) % num_nodes]
        cycle_edges.add((min(u,v), max(u,v)))

    return G, cycle_edges

def build_pyg_data(G, cycle_edges):
    """Build a PyG Data object with node features, edge index, edge features, and edge labels."""
    # Sort the edges to have consistent ordering
    sorted_edges = sorted(list(G.edges()))

    # Node features (degree, for instance)
    degrees = [G.degree(n) for n in G.nodes()]
    x = torch.tensor(degrees, dtype=torch.float).view(-1, 1)  # shape [num_nodes, 1]

    # Build edge index
    edge_index = torch.tensor([[u, v] for (u, v) in sorted_edges], dtype=torch.long).t()  # shape [2, num_edges]

    # Edge features: e.g., sum of node degrees
    edge_feats = []
    edge_labels = []
    n = G.number_of_nodes()
    for (u, v) in sorted_edges:
        sum_deg = G.degree(u) + G.degree(v)
        # Possibly normalized sum
        feat = [sum_deg / n]
        edge_feats.append(feat)

        # Label = 1 if edge is in the forced cycle
        label = 1 if (min(u,v), max(u,v)) in cycle_edges else 0
        edge_labels.append(label)

    edge_attr = torch.tensor(edge_feats, dtype=torch.float)  # shape [num_edges, num_edge_features]
    edge_labels = torch.tensor(edge_labels, dtype=torch.long) # shape [num_edges]

    data = Data(
        x=x,
        edge_index=edge_index,
        edge_attr=edge_attr,
        y=edge_labels  # For edge classification, we often store these separately
    )
    return data


# Example usage
num_graphs = 10
dataset = []
for _ in range(num_graphs):
    G, cycle_edges = generate_random_graph_with_cycle(num_nodes=15)
    data_obj = build_pyg_data(G, cycle_edges)
    dataset.append(data_obj)

loader = DataLoader(dataset, batch_size=2, shuffle=True)


In [2]:
pip install torch-geometric


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.5 MB/s eta 0:00:00
